In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np

C:\Users\nimam\AppData\Local\Temp\ipykernel_16460\1438236441.py:2: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


In [ ]:
pip uninstall pyproj

In [2]:
#pip install geopandas


In [3]:
 #pip install shapely

# Yeild Data

In [5]:
df = pd.read_csv('C:/Users/nimam/OneDrive/Desktop/oon laptop/New folder/New folder/Long-Term-Crop-Yields-NUTS3-Germany.csv',
                 sep=';', encoding= 'unicode_escape')
print(df.columns)
print ('******************************************')
df['Nuts3_name'] = df['Nuts3_name'].str.strip() # to remove space from the string
print (df.head(20))
print ('******************************************')
df['NUTS_CODE'] = df['Nuts3_code'] # later, we use these column for merging
print (df)
print ('******************************************')
df_null = df[df['Nuts3_code'].isna()] # to see how many of nuts does not have code
pd.unique(df_null['Nuts3_name']) # there are two counties without code

# print (df.loc[df['Nuts3_name'] == 'Flensburg, Kreisfreie Stadt'])

Index(['FedState', 'Nuts3_code', 'Nuts3_name', 'crop', 'year', 'yield'], dtype='object')
******************************************
             FedState Nuts3_code                   Nuts3_name           crop  \
0   SchleswigHolstein      DEF01  Flensburg, Kreisfreie Stadt   winter wheat   
1   SchleswigHolstein      DEF01  Flensburg, Kreisfreie Stadt  winter barley   
2   SchleswigHolstein      DEF01  Flensburg, Kreisfreie Stadt  winter canola   
3   SchleswigHolstein      DEF01  Flensburg, Kreisfreie Stadt   silage maize   
4   SchleswigHolstein      DEF01  Flensburg, Kreisfreie Stadt   winter wheat   
5   SchleswigHolstein      DEF01  Flensburg, Kreisfreie Stadt  winter barley   
6   SchleswigHolstein      DEF01  Flensburg, Kreisfreie Stadt  winter canola   
7   SchleswigHolstein      DEF01  Flensburg, Kreisfreie Stadt   silage maize   
8   SchleswigHolstein      DEF01  Flensburg, Kreisfreie Stadt   winter wheat   
9   SchleswigHolstein      DEF01  Flensburg, Kreisfreie Stadt  winte

array(['Osterode am Harz, Landkreis', 'Hannover, Landeshauptstadt'],
      dtype=object)

In [6]:
# Osterode am Harz, Landkreis:
#on 1 November 2016, Osterode ceased to become a seperate district and was merged with an enlarged Göttingen.
#There is data for crop yeilds from 1996 to 2016, but very similar to Göttingen.On the other hand, the zeild data 
#for Göttingen does not show a big change after 2016, while the Osterode zeild data should be added. So, we can
#say that the data from 1996 to 2016 are also already merged and we can skip and drop Ostrode data. The Göttingen 
#code is DE91C.
# Hannover, Landeshauptstadt:
#it is a location inside Hannover city center and from 1999 there is no wield data for this region (except 2018),
#So we can simply ingnore and drop this region. The whole area is called Hannover Landkreis with the code DE929.

df = df.dropna(subset=['Nuts3_code']).reset_index(drop=True) # drop nuts without code
print (df)
print ('******************************************')
df_null = df[df['Nuts3_code'].isna()] 
print (len(df_null))
print ('******************************************')
pd.unique(df_null['Nuts3_name'])
print (len(pd.unique(df['Nuts3_code'])))
print ('******************************************')
df.isna().sum()

                FedState Nuts3_code                   Nuts3_name  \
0      SchleswigHolstein      DEF01  Flensburg, Kreisfreie Stadt   
1      SchleswigHolstein      DEF01  Flensburg, Kreisfreie Stadt   
2      SchleswigHolstein      DEF01  Flensburg, Kreisfreie Stadt   
3      SchleswigHolstein      DEF01  Flensburg, Kreisfreie Stadt   
4      SchleswigHolstein      DEF01  Flensburg, Kreisfreie Stadt   
...                  ...        ...                          ...   
32155          Thüringen      DEG0I    SaalfeldRudolstadt, Kreis   
32156          Thüringen      DEG0J           SaaleHolzlandKreis   
32157          Thüringen      DEG0K               SaaleOrlaKreis   
32158          Thüringen      DEG0L                 Greiz, Kreis   
32159          Thüringen      DEG0M      Altenburger Land, Kreis   

                crop  year  yield NUTS_CODE  
0       winter wheat  2019    NaN     DEF01  
1      winter barley  2019    NaN     DEF01  
2      winter canola  2019    NaN     DEF01  

FedState         0
Nuts3_code       0
Nuts3_name       0
crop             0
year             0
yield         2394
NUTS_CODE        0
dtype: int64

# Geometry Data

In [9]:
fp0 = "C:/Users/nimam/OneDrive/Desktop/oon laptop/New folder/New folder/nuts250_0101/250_NUTS3.shp"
print (fp0)

C:/Users/nimam/OneDrive/Desktop/oon laptop/New folder/New folder/nuts250_0101/250_NUTS3.shp


In [10]:
data0 = gpd.read_file(fp0)
print (data0['NUTS_CODE'])
print ('******************************************')
print (data0)
print ('******************************************')
print (len(pd.unique(data0['NUTS_CODE']))) # there is 428 rows, while there are 401 unique counties. 
print ('******************************************')
duplicateRows = data0[data0.duplicated(['NUTS_CODE'],keep='last')] # So, there are 27 duplicated county.
duplicateRows
#there are some duplicated nuts' codes in the shapefile attribute table. So, using QGIS, the similar codes have been 
#dissolved geometrically and from now on, the new dissolved shapefile will be used.

0      DE111
1      DE112
2      DE113
3      DE114
4      DE115
       ...  
423    DEF09
424    DEF0A
425    DEF0B
426    DEF0C
427    DEF0E
Name: NUTS_CODE, Length: 428, dtype: object
******************************************
     GF  NUTS_LEVEL NUTS_CODE              NUTS_NAME  \
0     4           3     DE111  Stuttgart, Stadtkreis   
1     4           3     DE112              Böblingen   
2     4           3     DE113              Esslingen   
3     4           3     DE114              Göppingen   
4     4           3     DE115            Ludwigsburg   
..   ..         ...       ...                    ...   
423   2           3     DEF09              Pinneberg   
424   2           3     DEF0A                   Plön   
425   2           3     DEF0B  Rendsburg-Eckernförde   
426   2           3     DEF0C    Schleswig-Flensburg   
427   2           3     DEF0E              Steinburg   

                                              geometry  
0    POLYGON ((516514.129 5412584.910, 5

,GF,NUTS_LEVEL,NUTS_CODE,NUTS_NAME,geometry
32,4,3,DE138,Konstanz,"MULTIPOLYGON (((477594.656 5284692.579, 477721..."
159,4,3,DE501,"Bremen, Kreisfreie Stadt","MULTIPOLYGON (((470069.147 5937300.563, 471086..."
160,4,3,DE502,"Bremerhaven, Kreisfreie Stadt","MULTIPOLYGON (((469529.375 5939817.331, 470542..."
161,4,3,DE600,Hamburg,"MULTIPOLYGON (((467342.107 5975636.297, 467364..."
188,4,3,DE803,"Rostock, Kreisfreie Stadt","MULTIPOLYGON (((702598.302 6006620.218, 702532..."
191,4,3,DE80K,Landkreis Rostock,"MULTIPOLYGON (((711443.180 5948511.888, 710374..."
192,4,3,DE80L,Vorpommern-Rügen,"MULTIPOLYGON (((769820.277 6019736.746, 769828..."
193,4,3,DE80M,Nordwestmecklenburg,"MULTIPOLYGON (((650647.730 5946861.740, 651034..."
194,4,3,DE80N,Vorpommern-Greifswald,"MULTIPOLYGON (((817824.528 5945144.240, 817774..."
214,4,3,DE932,Cuxhaven,"POLYGON ((496649.833 5965309.608, 496892.511 5..."


In [11]:
# this cell is similar to the above, just using the dissolved shapefile.
fp = "C:/Users/nimam/OneDrive/Desktop/oon laptop/New folder/New folder/dissolved nuts_3/dissolved_nuts_3.shp"
print (fp)
data = gpd.read_file(fp)
print (data['NUTS_CODE'])
print ('******************************************')
print (data)
print ('******************************************')
print (len(pd.unique(data['NUTS_CODE'])))
print ('******************************************')
duplicateRows = data[data.duplicated(['NUTS_CODE'],keep='last')]
print (duplicateRows)
print ('******************************************')
print (data.isna().sum())
print ('******************************************')
print (len(pd.unique(data['NUTS_NAME']))) # there is no duplicated nuts name in the data
print ('******************************************')
data.plot()

C:/Users/nimam/OneDrive/Desktop/oon laptop/New folder/New folder/dissolved nuts_3/dissolved_nuts_3.shp
0      DE732
1      DEA15
2      DED21
3      DEA14
4      DEA17
       ...  
396    DE932
397    DE731
398    DE935
399    DE934
400    DE733
Name: NUTS_CODE, Length: 401, dtype: object
******************************************
     GF  NUTS_LEVEL NUTS_CODE                          NUTS_NAME  \
0     4           3     DE732                              Fulda   
1     4           3     DEA15  Mönchengladbach, Kreisfreie Stadt   
2     4           3     DED21          Dresden, Kreisfreie Stadt   
3     4           3     DEA14          Krefeld, Kreisfreie Stadt   
4     4           3     DEA17       Oberhausen, Kreisfreie Stadt   
..   ..         ...       ...                                ...   
396   4           3     DE932                           Cuxhaven   
397   4           3     DE731           Kassel, Kreisfreie Stadt   
398   4           3     DE935                Lüneburg, 

# Merged Data

In [12]:
merged = data.merge(df, on='NUTS_CODE')
# print (merged.loc[merged['Nuts3_name'] == 'St Wendel, Landkreis'])
# print ('******************************************')
print (merged)
print ('******************************************')
print (len(pd.unique(merged['NUTS_NAME'])))
print ('******************************************')
merged.isna().sum()

       GF  NUTS_LEVEL NUTS_CODE           NUTS_NAME  \
0       4           3     DE732               Fulda   
1       4           3     DE732               Fulda   
2       4           3     DE732               Fulda   
3       4           3     DE732               Fulda   
4       4           3     DE732               Fulda   
...    ..         ...       ...                 ...   
32155   4           3     DE733  Hersfeld-Rotenburg   
32156   4           3     DE733  Hersfeld-Rotenburg   
32157   4           3     DE733  Hersfeld-Rotenburg   
32158   4           3     DE733  Hersfeld-Rotenburg   
32159   4           3     DE733  Hersfeld-Rotenburg   

                                                geometry FedState Nuts3_code  \
0      POLYGON ((554454.057 5628755.874, 554511.010 5...   Hessen      DE732   
1      POLYGON ((554454.057 5628755.874, 554511.010 5...   Hessen      DE732   
2      POLYGON ((554454.057 5628755.874, 554511.010 5...   Hessen      DE732   
3      POLYGON ((55

GF               0
NUTS_LEVEL       0
NUTS_CODE        0
NUTS_NAME        0
geometry         0
FedState         0
Nuts3_code       0
Nuts3_name       0
crop             0
year             0
yield         2394
dtype: int64

In [13]:
a = merged['NUTS_CODE'].value_counts()
pd.unique(a).sum() # all the nuts3 codes have 96 yeild data. 24 years * 4 crops = 96
                   # 96 * 335 counties code = 32160

96

In [14]:
merged.dtypes

GF               int64
NUTS_LEVEL       int64
NUTS_CODE       object
NUTS_NAME       object
geometry      geometry
FedState        object
Nuts3_code      object
Nuts3_name      object
crop            object
year             int64
yield          float64
dtype: object

In [15]:
# just to check if all the counties that exist in the shapefile but not in the yield dataset have been droped afer merging.
# a=[]
# for i in data['NUTS_CODE']:
#     if i not in list(df['NUTS_CODE']):
#         a.append(i)
# print (len(a))

# c=[]
# for i in data['NUTS_CODE']:
#     if i not in list(merged['NUTS_CODE']):
#         c.append(i)
# print (len(c))

# d=[]
# for i in a:
#     if i not in c:
#         d.append(i)
# print (len(d))

In [16]:
# to drop data before 2000, the year that MODIS data are available.
unwanted_years = [1996, 1997, 1998, 1999, 2000]
merged_1 = merged[merged.year.isin(unwanted_years) == False].reset_index(drop=True) 
merged_1

,GF,NUTS_LEVEL,NUTS_CODE,NUTS_NAME,geometry,FedState,Nuts3_code,Nuts3_name,crop,year,yield
0,4,3,DE732,Fulda,"POLYGON ((554454.057 5628755.874, 554511.010 5...",Hessen,DE732,"Fulda, Landkreis",winter wheat,2019,79.0
1,4,3,DE732,Fulda,"POLYGON ((554454.057 5628755.874, 554511.010 5...",Hessen,DE732,"Fulda, Landkreis",winter barley,2019,66.0
2,4,3,DE732,Fulda,"POLYGON ((554454.057 5628755.874, 554511.010 5...",Hessen,DE732,"Fulda, Landkreis",winter canola,2019,26.0
3,4,3,DE732,Fulda,"POLYGON ((554454.057 5628755.874, 554511.010 5...",Hessen,DE732,"Fulda, Landkreis",silage maize,2019,448.0
4,4,3,DE732,Fulda,"POLYGON ((554454.057 5628755.874, 554511.010 5...",Hessen,DE732,"Fulda, Landkreis",winter wheat,2018,71.0
...,...,...,...,...,...,...,...,...,...,...,...
25455,4,3,DE733,Hersfeld-Rotenburg,"POLYGON ((571913.734 5635252.884, 571891.950 5...",Hessen,DE733,"HersfeldRotenburg, Landkreis",silage maize,2002,441.0
25456,4,3,DE733,Hersfeld-Rotenburg,"POLYGON ((571913.734 5635252.884, 571891.950 5...",Hessen,DE733,"HersfeldRotenburg, Landkreis",winter wheat,2001,75.0
25457,4,3,DE733,Hersfeld-Rotenburg,"POLYGON ((571913.734 5635252.884, 571891.950 5...",Hessen,DE733,"HersfeldRotenburg, Landkreis",winter barley,2001,62.0
25458,4,3,DE733,Hersfeld-Rotenburg,"POLYGON ((571913.734 5635252.884, 571891.950 5...",Hessen,DE733,"HersfeldRotenburg, Landkreis",winter canola,2001,33.0


In [12]:
# to drop useless columns
merged_2 = merged_1.drop(columns = ['GF', 'NUTS_LEVEL', 'Nuts3_code', 'Nuts3_name'])
merged_2

,NUTS_CODE,NUTS_NAME,geometry,FedState,crop,year,yield
0,DE732,Fulda,"POLYGON ((554454.057 5628755.874, 554511.010 5...",Hessen,winter wheat,2019,79.0
1,DE732,Fulda,"POLYGON ((554454.057 5628755.874, 554511.010 5...",Hessen,winter barley,2019,66.0
2,DE732,Fulda,"POLYGON ((554454.057 5628755.874, 554511.010 5...",Hessen,winter canola,2019,26.0
3,DE732,Fulda,"POLYGON ((554454.057 5628755.874, 554511.010 5...",Hessen,silage maize,2019,448.0
4,DE732,Fulda,"POLYGON ((554454.057 5628755.874, 554511.010 5...",Hessen,winter wheat,2018,71.0
...,...,...,...,...,...,...,...
25455,DE733,Hersfeld-Rotenburg,"POLYGON ((571913.734 5635252.884, 571891.950 5...",Hessen,silage maize,2002,441.0
25456,DE733,Hersfeld-Rotenburg,"POLYGON ((571913.734 5635252.884, 571891.950 5...",Hessen,winter wheat,2001,75.0
25457,DE733,Hersfeld-Rotenburg,"POLYGON ((571913.734 5635252.884, 571891.950 5...",Hessen,winter barley,2001,62.0
25458,DE733,Hersfeld-Rotenburg,"POLYGON ((571913.734 5635252.884, 571891.950 5...",Hessen,winter canola,2001,33.0


In [ ]:
# to write and save a shapefile (extract a shape file)
merged_2.to_file(filename = 'C:/Users/mohammadigolafshani/Desktop/New folder/New folder/myshapefile_2001',
               driver='ESRI Shapefile')


In [13]:
pd.unique(merged_2['FedState'])

array(['Hessen', 'Sachsen', 'Niedersachsen', 'Saarland', 'Brandenburg',
       'NordrheinWestfalen', 'RheinlandPfalz', 'Bayern',
       'BadenWürttemberg', 'Thüringen', 'SachsenAnhalt',
       'MecklenburgVorpommern', 'SchleswigHolstein'], dtype=object)

In [14]:
merged_3 = merged_2[merged_2['FedState'].isin(['Brandenburg','Sachsen'])]
print (pd.unique(merged_3['FedState']))
merged_3

['Sachsen' 'Brandenburg']


,NUTS_CODE,NUTS_NAME,geometry,FedState,crop,year,yield
76,DED21,"Dresden, Kreisfreie Stadt","POLYGON ((832372.516 5680250.342, 832619.681 5...",Sachsen,winter wheat,2019,77.0
77,DED21,"Dresden, Kreisfreie Stadt","POLYGON ((832372.516 5680250.342, 832619.681 5...",Sachsen,winter barley,2019,NaN
78,DED21,"Dresden, Kreisfreie Stadt","POLYGON ((832372.516 5680250.342, 832619.681 5...",Sachsen,winter canola,2019,38.0
79,DED21,"Dresden, Kreisfreie Stadt","POLYGON ((832372.516 5680250.342, 832619.681 5...",Sachsen,silage maize,2019,330.0
80,DED21,"Dresden, Kreisfreie Stadt","POLYGON ((832372.516 5680250.342, 832619.681 5...",Sachsen,winter wheat,2018,71.0
...,...,...,...,...,...,...,...
24543,DED2F,Sächsische Schweiz-Osterzgebirge,"POLYGON ((867990.571 5650373.037, 868087.194 5...",Sachsen,silage maize,2002,421.0
24544,DED2F,Sächsische Schweiz-Osterzgebirge,"POLYGON ((867990.571 5650373.037, 868087.194 5...",Sachsen,winter wheat,2001,65.0
24545,DED2F,Sächsische Schweiz-Osterzgebirge,"POLYGON ((867990.571 5650373.037, 868087.194 5...",Sachsen,winter barley,2001,66.0
24546,DED2F,Sächsische Schweiz-Osterzgebirge,"POLYGON ((867990.571 5650373.037, 868087.194 5...",Sachsen,winter canola,2001,36.0


In [15]:
merged_4 = merged_2.loc[merged_2['FedState'] == 'NordrheinWestfalen'] 
merged_4

,NUTS_CODE,NUTS_NAME,geometry,FedState,crop,year,yield
2964,DEA2B,Rheinisch-Bergischer Kreis,"POLYGON ((369232.192 5642545.606, 369222.953 5...",NordrheinWestfalen,winter wheat,2019,NaN
2965,DEA2B,Rheinisch-Bergischer Kreis,"POLYGON ((369232.192 5642545.606, 369222.953 5...",NordrheinWestfalen,winter barley,2019,NaN
2966,DEA2B,Rheinisch-Bergischer Kreis,"POLYGON ((369232.192 5642545.606, 369222.953 5...",NordrheinWestfalen,winter canola,2019,NaN
2967,DEA2B,Rheinisch-Bergischer Kreis,"POLYGON ((369232.192 5642545.606, 369222.953 5...",NordrheinWestfalen,silage maize,2019,NaN
2968,DEA2B,Rheinisch-Bergischer Kreis,"POLYGON ((369232.192 5642545.606, 369222.953 5...",NordrheinWestfalen,winter wheat,2018,NaN
...,...,...,...,...,...,...,...
15651,DEA5B,Soest,"POLYGON ((444505.568 5697079.226, 444385.760 5...",NordrheinWestfalen,silage maize,2002,466.0
15652,DEA5B,Soest,"POLYGON ((444505.568 5697079.226, 444385.760 5...",NordrheinWestfalen,winter wheat,2001,95.0
15653,DEA5B,Soest,"POLYGON ((444505.568 5697079.226, 444385.760 5...",NordrheinWestfalen,winter barley,2001,81.0
15654,DEA5B,Soest,"POLYGON ((444505.568 5697079.226, 444385.760 5...",NordrheinWestfalen,winter canola,2001,38.0


In [16]:
merged_5 = merged_2[merged_2['FedState'].isin(['MecklenburgVorpommern', 'SchleswigHolstein'])]
print (pd.unique(merged_4['FedState']))
merged_5

['NordrheinWestfalen']


,NUTS_CODE,NUTS_NAME,geometry,FedState,crop,year,yield
17176,DE80M,Nordwestmecklenburg,"MULTIPOLYGON (((651417.472 5946020.111, 651664...",MecklenburgVorpommern,winter wheat,2019,86.0
17177,DE80M,Nordwestmecklenburg,"MULTIPOLYGON (((651417.472 5946020.111, 651664...",MecklenburgVorpommern,winter barley,2019,89.0
17178,DE80M,Nordwestmecklenburg,"MULTIPOLYGON (((651417.472 5946020.111, 651664...",MecklenburgVorpommern,winter canola,2019,38.0
17179,DE80M,Nordwestmecklenburg,"MULTIPOLYGON (((651417.472 5946020.111, 651664...",MecklenburgVorpommern,silage maize,2019,370.0
17180,DE80M,Nordwestmecklenburg,"MULTIPOLYGON (((651417.472 5946020.111, 651664...",MecklenburgVorpommern,winter wheat,2018,59.0
...,...,...,...,...,...,...,...
22567,DEF01,"Flensburg, Kreisfreie Stadt","POLYGON ((526547.941 6074977.321, 526560.310 6...",SchleswigHolstein,silage maize,2002,364.0
22568,DEF01,"Flensburg, Kreisfreie Stadt","POLYGON ((526547.941 6074977.321, 526560.310 6...",SchleswigHolstein,winter wheat,2001,97.0
22569,DEF01,"Flensburg, Kreisfreie Stadt","POLYGON ((526547.941 6074977.321, 526560.310 6...",SchleswigHolstein,winter barley,2001,88.0
22570,DEF01,"Flensburg, Kreisfreie Stadt","POLYGON ((526547.941 6074977.321, 526560.310 6...",SchleswigHolstein,winter canola,2001,39.0


In [17]:
merged_6 = merged_2.loc[merged_2['FedState'] == 'Niedersachsen'] 
merged_6.reset_index(drop=True, inplace=True)
merged_6_1 = merged_6[:1520]
merged_6_2 = merged_6[1520:]
#merged_6

In [18]:
merged_7 = merged_2.loc[merged_2['FedState'] == 'SachsenAnhalt'] 
merged_7

,NUTS_CODE,NUTS_NAME,geometry,FedState,crop,year,yield
13072,DEE0B,Saalekreis,"MULTIPOLYGON (((667788.479 5690399.571, 667553...",SachsenAnhalt,winter wheat,2019,52.0
13073,DEE0B,Saalekreis,"MULTIPOLYGON (((667788.479 5690399.571, 667553...",SachsenAnhalt,winter barley,2019,62.0
13074,DEE0B,Saalekreis,"MULTIPOLYGON (((667788.479 5690399.571, 667553...",SachsenAnhalt,winter canola,2019,27.0
13075,DEE0B,Saalekreis,"MULTIPOLYGON (((667788.479 5690399.571, 667553...",SachsenAnhalt,silage maize,2019,214.0
13076,DEE0B,Saalekreis,"MULTIPOLYGON (((667788.479 5690399.571, 667553...",SachsenAnhalt,winter wheat,2018,60.0
...,...,...,...,...,...,...,...
17627,DEE01,"Dessau-Roßlau, Kreisfreie Stadt","MULTIPOLYGON (((725145.173 5762806.069, 725380...",SachsenAnhalt,silage maize,2002,NaN
17628,DEE01,"Dessau-Roßlau, Kreisfreie Stadt","MULTIPOLYGON (((725145.173 5762806.069, 725380...",SachsenAnhalt,winter wheat,2001,NaN
17629,DEE01,"Dessau-Roßlau, Kreisfreie Stadt","MULTIPOLYGON (((725145.173 5762806.069, 725380...",SachsenAnhalt,winter barley,2001,NaN
17630,DEE01,"Dessau-Roßlau, Kreisfreie Stadt","MULTIPOLYGON (((725145.173 5762806.069, 725380...",SachsenAnhalt,winter canola,2001,NaN


In [ ]:
# to write and save a shapefile (extract a shape file)
merged_3.to_file(filename = 'C:/Users/mohammadigolafshani/Desktop/New folder/New folder/NorthGermany1_shapefile_2001',
               driver='ESRI Shapefile')

In [ ]:
merged_4.to_file(filename = 'C:/Users/mohammadigolafshani/Desktop/New folder/New folder/NorthGermany2_shapefile_2001',
               driver='ESRI Shapefile')

In [ ]:
merged_5.to_file(filename = 'C:/Users/mohammadigolafshani/Desktop/New folder/New folder/NorthGermany3_shapefile_2001',
               driver='ESRI Shapefile')

In [ ]:
merged_6_1.to_file(filename = 'C:/Users/mohammadigolafshani/Desktop/New folder/New folder/NorthGermany4_shapefile_2001',
               driver='ESRI Shapefile')

In [ ]:
merged_6_2.to_file(filename = 'C:/Users/mohammadigolafshani/Desktop/New folder/New folder/NorthGermany5_shapefile_2001',
               driver='ESRI Shapefile')

In [ ]:
merged_7.to_file(filename = 'C:/Users/mohammadigolafshani/Desktop/New folder/New folder/NorthGermany6_shapefile_2001',
               driver='ESRI Shapefile')

In [27]:
merged_4.isna().sum()

NUTS_CODE      0
NUTS_NAME      0
geometry       0
FedState       0
crop           0
year           0
yield        226
dtype: int64

In [ ]:
nan = merged_3[merged_3['yield'].isna()]

In [ ]:
dups_color_and_shape = nan.pivot_table(columns=['FedState'], aggfunc='size')
dups_color_and_shape

In [ ]:
dups_color_and_shape1 = nan.pivot_table(columns=['FedState','NUTS_NAME'], aggfunc='size')
dups_color_and_shape1